In [5]:
import uuid
import pandas as pd


In [6]:
df = pd.read_table('../data/20240529_all_procedures.rpt')

In [7]:
df.columns


#    'identifier_value', 'subject_Patient_value', 'specialty_code_original', 2
#    'surgery_admission_start', 'surgery_admission_end', 'time_OR', 'name', 6
#    'postop_diagnosis', 'postop_diagnosis_code', 'postop_diagnosis_text', 9
#    'procedure_code', 'procedure_text', 'main_procedure', 'procedure_start',13
#    'procedure_end', 'procedure_duration', 'procedure_duration_fix', 16
#    'age_procedure', 'status_sternum', 'ECC_duur_min', 'AoX_duur_min',20
#    'Circulatory_arrest_min', 'Antegrade_circulation_min' 22
# ['identifier_value', 'subject_Patient_value',
#   'time_OR', 'postop_diagnosis_code', 
#   'postop_diagnosis_text', 'main_procedure',
#   'procedure_duration', 'procedure_duration_fix',
#   'age_procedure', 'status_sternum', 'ECC_duur_min', 
#   'AoX_duur_min','Circulatory_arrest_min', 
#   'Antegrade_circulation_min' ]

Index(['identifier_value', 'subject_Patient_value', 'specialty_code_original',
       'surgery_admission_start', 'surgery_admission_end', 'time_OR', 'name',
       'postop_diagnosis', 'postop_diagnosis_code', 'postop_diagnosis_text',
       'procedure_code', 'procedure_text', 'main_procedure', 'procedure_start',
       'procedure_end', 'procedure_duration', 'procedure_duration_fix',
       'age_procedure', 'status_sternum', 'ECC_duur_min', 'AoX_duur_min',
       'Circulatory_arrest_min', 'Antegrade_circulation_min'],
      dtype='object')

### Trial and Erros 

In [8]:
x = uuid.uuid4().int

In [9]:
df = pd.read_excel('20240522_ctc_procedures.xlsx', header=None)

FileNotFoundError: [Errno 2] No such file or directory: '20240522_ctc_procedures.xlsx'

In [ ]:
df.head()

In [ ]:
pid = df[1].unique()

In [ ]:
pid = df[1].unique()
patientId = []
studyId = []

for x in pid: 
    patientId.append(x)
    studyId.append(uuid.uuid4().int)

if len(studyId) == len(pid):
    print('No copies, al good to continue')
    df_studyId = dict(zip(patientId, studyId))
    print(df_studyId)
    print(df[1].head())
    df[1].replace(df_studyId,inplace=True)
    print(df[1].head())
else: 
    print('WARNING, there are duplicates in the studyId list')

# df_studyId = pd.DataFrame({'patientId':patientId, 'studyId': studyId})

# df_studyId.to_csv('pID_stID_keys.csv')

In [ ]:
df.head(2)
df2 = df[[1,7,8,9,10,13,14,15,16,17,18,19]]

In [ ]:
df2.to_csv('20240522_2_ctc_procedure_anonymised.csv', header=None,index=None)

In [ ]:
df2.to_excel('20240522_2_ctc_procedure_anonymised.xlsx', header=None, index=None)


In [ ]:
df_check = pd.read_excel('20240522_2_ctc_procedure_anonymised.xlsx')

In [ ]:
df_check = pd.read_csv('20240522_2_ctc_procedure_anonymised.csv', header=None, index_col=None)

In [ ]:
df_check.head()

In [ ]:
df_check.info()

In [ ]:
a = dict(df_check[4].value_counts())

In [ ]:
df_single = df_check[4]

In [ ]:
test = pd.DataFrame(df_single.value_counts(), columns= None)

In [ ]:
test.tail(30)

In [ ]:
value_counts = df_check[3].value_counts()
filtered_df = df_check[df_check[3].isin(value_counts[value_counts > 5].index)]

In [ ]:
procedures_toexclude = '989999'
final_df = filtered_df[filtered_df[3] != procedures_toexclude]

In [ ]:
value_counts2 = pd.DataFrame(final_df[4].value_counts())


In [ ]:
value_counts2.tail(23)

### Remove negative procedure durations with fix 

In [ ]:
import pandas as pd
import numpy as np
df_raw = pd.read_table('data/20240528_all_procedures.rpt')

df = pd.read_table('data/20240528_all_procedures.rpt')
df['procedure_duration'] = df[['time_OR', 'procedure_duration', 'procedure_duration_fix']].replace(0, np.nan).abs().min(axis=1)
df = df.drop(columns=['procedure_duration_fix'])

value_counts = df['postop_diagnosis_code'].value_counts()
filter1_df = df[df['postop_diagnosis_code'].isin(value_counts[value_counts >= 5].index)]


In [ ]:
filter1_df = df[df['procedure_duration'].isin(value_counts[value_counts > 5].index)]

In [ ]:
df.loc[df['procedure_duration'] <= 0, 'procedure_duration'] = df.loc[df['procedure_duration'] < 0, 'procedure_duration_fix']

In [ ]:
mask = df['procedure_duration'] <= 0
df.loc[mask, 'procedure_duration'] = df.loc[mask, 'procedure_duration_fix']

In [ ]:
df['procedure_duration'].value_counts()

# Fixed code
To create function


In [ ]:
df_test= pd.read_table('data/20240528_all_procedures.rpt')

In [ ]:
df_test['postop_diagnosis_text'].value_counts().reset_index().to_csv('df.csv')

In [ ]:
unique_diagnoses = pd.read_csv('df.csv')

### FUNCTION 
Criteria to take into account 
- Data (CSV) must contain headers (in same format as SQL & Documentation)
- Filters are 'toggleable'


!TODO: 
- Update USECOLS in read
- 

In [ ]:
import uuid
import pandas as pd 
import numpy as np

def create_unidentifiable_codes(input_path, output_path_data, keyfile_pid_path, keyfile_encounter_path, filter1=True, filter2=True):
    df_raw = pd.read_table(input_path, usecols=['identifier_value', 'subject_Patient_value', 'time_OR', 'postop_diagnosis_code','postop_diagnosis_text', 'procedure_code', 'procedure_text', 'procedure_duration', 'procedure_duration_fix','age_procedure', 'status_sternum', 'ECC_duur_min', 'AoX_duur_min', 'Circulatory_arrest_min', 'Antegrade_circulation_min'])
    
    #PatientID section 
    pid = df_raw['subject_Patient_value'].unique()
    patientId = []
    studyId = []

    for x in pid: 
        patientId.append(x)
        studyId.append(uuid.uuid4().hex)

    if len(set(studyId)) == len(set(pid)):              # check for number of unique values, should be equal
        print('No copies, al good to continue')         # otherwise same studyID generated (very unlikely)
        
        keyfile = dict(zip(patientId, studyId))      
        df_raw['subject_Patient_value'].replace(keyfile, inplace=True)         # replace patientID with studyID

        pd.DataFrame({'patientId':patientId, 'studyId': studyId}).to_pickle(keyfile_pid_path)     #('pID_stID_keys.pkl')
    else: 
        print('WARNING, there are duplicates in the studyId list')

    # Encounter section
    encounter = df_raw['identifier_value'].unique()

    encounter_value = []
    ok_value = []

    for enc in encounter: 
        encounter_value.append(enc)
        ok_value.append(uuid.uuid4().hex)

    if len(set(ok_value)) == len(set(encounter)):               # check for number of unique values, should be equal
        print('No copies, al good to continue')                 # otherwise same ok_value generated (very unlikely)
        
        keyfile_encounter = dict(zip(encounter_value, ok_value))      
        df_raw['identifier_value'].replace(keyfile_encounter, inplace=True)         # replace patientID with studyID

        pd.DataFrame({'encounter_value':patientId, 'ok_value': studyId}).to_pickle(keyfile_encounter_path)    #('enc_ok_value_keys.pkl')
    else: 
        print('WARNING, there are duplicates in the studyId list')

    df_raw['procedure_duration'] = df_raw[['time_OR', 'procedure_duration', 'procedure_duration_fix']].replace(0, np.nan).abs().min(axis=1)
    df = df_raw.drop(columns=['procedure_duration_fix'])

    # filter1 = remove rare postop-diagnoses
    if filter1: 
        value_counts = df['postop_diagnosis_code'].value_counts()
        filter1_df = df[df['postop_diagnosis_code'].isin(value_counts[value_counts > 5].index)]

    #standardizing dtypes 
    if filter2:
        filter1_df['procedure_code'].astype(str) # NO EFFECT ? --> perhaps with loading data 

        procedures_toexclude = [989999]             # Geen procedure
        for exclude in procedures_toexclude:
            filter2_df = filter1_df[filter1_df['procedure_code'] != exclude]

    filter2_df.to_csv(output_path_data, header=True, index=False)

### First use of function

In [ ]:
create_unidentifiable_codes('data/20240528_all_procedures.rpt', 'test/20240528_filter.csv', 'test/20240528_keyfile_pid.pkl', 'test/20240528_keyfile_identifier.pkl', filter1=True, filter2=True)


In [ ]:
df_test = pd.read_csv('test/20240528_filter.csv')

### continue normal code


In [ ]:
import pandas as pd 

df_test = pd.read_csv('20240522_3_ctc_procedure_anonymised.csv')

In [ ]:
df_keys = pd.read_pickle('pID_stID_keys.pkl')

## check new data extraction

In [ ]:
df = pd.read_excel('data/20240523_procedure.xlsx')

In [ ]:
df['postop_diagnosis_text'].value_counts()

In [ ]:
df['postop_diagnosis_code'].value_counts()


### Test report files


In [ ]:
df = pd.read_table('data/20240528_all_procedures.rpt')

In [ ]:
df.head()